In [14]:
# uradicemo jednu linearnu regresiju nad nekim podacima. To je algoritam nadgeldanog ucenja
# pokusavamo da predvidimo neku ciljanu vrednost

# pipeline koji je vrlo cest kod problema masinskog ucenja:
# 1) Priprema i obrada podataka
# 2) Treniranje modela
# 3) Evaulacija modela
# 4) Koriscenje modela 

In [40]:
import pandas as pd
import numpy as np

# ne mroamo da pamtimo ove includove jer ce ih cesto biti gomila. na ispitu dobijamo verovatno vec gotove
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [18]:
# mpg je miles per galon, ovo cemo pokusavati da predvidimo
df = pd.read_csv('auto-mpg.csv')   
print(df.head())

    mpg  cylinders  displacement  horsepower  weight  acceleration  model-year
0  18.0          8         307.0       130.0    3504          12.0          70
1  15.0          8         350.0       165.0    3693          11.5          70
2  18.0          8         318.0       150.0    3436          11.0          70
3  16.0          8         304.0       150.0    3433          12.0          70
4  17.0          8         302.0       140.0    3449          10.5          70


In [20]:
# 1) prvo cemo obraditi podatke

# jedna jednostavna obrada koju mozemo da radimo je da nekako resimo nepostojece vrednosti. 
# cesto se u csv fajlovima nepostojece vrednosti oznace npr sa ? ili sa # ili praznim karakterom
# u nasem konkretnom csv-u nemamo nedostajuce vrednosti, ali radimo ovo za primer
# replace(sta_menjamo, cime_menjamo)
df = df.replace('?', np.nan)
# mi ovde npr izbacujemo sve nepostojece vrednosti
df = df.dropna()

In [25]:
# razdvojicemo podatke na ulazne podatke i na podatke koje hocemod a predvidimo
# ulazni podaci su nam svi oni bez miles per galon
X = df.drop('mpg', axis=1)
# a ovo su podaci koje predvidjamo
Y = df[['mpg']]

print(X.head())
print()
print(Y.head())

   cylinders  displacement  horsepower  weight  acceleration  model-year
0          8         307.0       130.0    3504          12.0          70
1          8         350.0       165.0    3693          11.5          70
2          8         318.0       150.0    3436          11.0          70
3          8         304.0       150.0    3433          12.0          70
4          8         302.0       140.0    3449          10.5          70

    mpg
0  18.0
1  15.0
2  18.0
3  16.0
4  17.0


In [29]:
# potrebno je podeliti podatke na trening i test skup, da se slucajno nebi desilo preprilagodjavanje
# (ne mora uvek nuzno na trening i test, ima i drugih podela, npr unakrsna validacija)
# mozemo ovo raditi rucno u pandasu ali u sklearn imamo vec ugradjenu funkciju za ovo

# test_size je u procentima koliko ukupnog skupa ce biti test skup 
# (alternativno,mogli smo da koristimo i train_size=...)
# random_state=... je seed za random brojeve na osnovu kojih ce se birati skupovi, ako ovo setujemo rucno,
# svaki put ce se raditi sa istim seedom, tj uvek ce biti slicna podela na train i test.
# ovo moze da nam bude korisno za testiranje promena koje napravimo 
# rezultat su podeljeni ulazni i izlazni podaci na train i test (naravno, podaci ce odgovarati bas
# onim podacima iz tabele, npr ako u train skupu treba da se nadje prvi red iz X, nacice se i prvi red iz Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [38]:
# 2) pravimo i treniramo model

# i za linearnu regresiju ne moramo sami da implementiramo nista, koristimo ugradjeno iz sklearn
# svi modeli iz biblioteke prate neku konvenciju. Svi imaju metode fit i predict
reg = LinearRegression()
# funkcija fit ce za nas napraviti neki model koji odgovara podacima naseg train skupa
reg.fit(X_train, Y_train)

# mozemo npr da vidimo slobodnog clana iz modela linearne regresije
print(reg.intercept_[0])

# nnpr da vidimo vrednosti modela dodeljene svakoj koloni
for idx, col_name in enumerate(X_train.columns):
    print(f'coef za {col_name} = {reg.coef_[0][idx]}')


-13.824169113397168
coef za cylinders = -0.2381301557148593
coef za displacement = 0.001683903382560182
coef za horsepower = -0.00025019058188301623
coef za weight = -0.006621560986020143
coef za acceleration = 0.010273669827510268
coef za model-year = 0.7625231149758138


In [45]:
# 3) evaluiramo model

# mozemo iz sklearn da koristimo vec neke ugradjene evaluacije npr r2 metriku i srednje kvadratnu gresku
# r2 metrika
r2_test = reg.score(X_test, Y_test)
r2_train = reg.score(X_train, Y_train)
print(f'r2 test {r2_test}')
print(f'r2 train {r2_train}')

# srednje kvadratna greska
# pazimo samo na argumente. Prvi argument su originalne vrednosti iz skupa, 
# a drugi treba da bude ono sto nas model predvidja
mse_test = mean_squared_error(Y_test, reg.predict(X_test))
mse_train = mean_squared_error(Y_train, reg.predict(X_train))
print(f'mse test {mse_test}')
print(f'mse train {mse_train}')

# mozemo saznati i za koliko u proseku milja po galonu gresimo tako sto uzmemo koren (jer je srednje kvadratna greska)
rmse_test = np.sqrt(mse_test)
rmse_train = np.sqrt(mse_train)
print(f'rmse test {rmse_test}')
print(f'rmse train {rmse_train}')

r2 test 0.7757849559316723
r2 train 0.8134917530842111
mse test 11.054880997716923
mse train 12.04121219273428
rmse test 3.3248881180750915
rmse train 3.4700449842522616


In [51]:
# 4) primer koriscenja modela

class MPGEvaluator:
    def __init__(self, model):
        self.model = model
        
    def predict(self, car_data):
        return self.model.predict(car_data)
    
mpgEvaluator = MPGEvaluator(reg)

# neki podatak za primer da vidimo kako ovo radi
# posto predvidjamo mpg, podaci ne treba da ga sadrze
# cylinders  displacement  horsepower  weight  acceleration  model-year
car_data = [8, 307.0, 130.0, 3504, 12.0, 70]
mpgEvaluator.predict([car_data])

array([[15.05317559]])